In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Detections

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])

    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [5]:
def calculate_statistics(landmarks, world_landmarks):
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_shoulder = (world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                 world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    world_left_ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
    world_left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
    right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_shoulder = (world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                  world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    world_right_ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
    world_right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    # Hips Below knees
    if landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y and landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y:
        hipsBelowKnees = True
    else:
        hipsBelowKnees = False

    # Weight shift (Forward/Backward)
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    ankle_mid_x = (left_ankle[0] + right_ankle[0]) / 2
    
    # Weight shift based on angle between toe heel ankle
    right_foot_angle = calculate_angle(right_foot_index, right_heel, right_ankle)
    left_foot_angle = calculate_angle(left_foot_index, left_heel, left_ankle)
    
    # Weight shift lateral
    left_mid_x = (left_hip[0] + left_ankle[0]) / 2

    metrics = {
    'depth_left': calculate_angle(left_ankle, left_knee, left_hip),
    
    'depth_right': calculate_angle(right_ankle, right_knee, right_hip),
    
    'knee_balance': [right_foot_angle, left_foot_angle],
    
    'hips_below_knees': hipsBelowKnees,
        
    'foot_distance': np.linalg.norm(np.array(world_left_ankle) - np.array(world_right_ankle)),
    
    'grip_width': np.linalg.norm(np.array(world_left_wrist) - np.array(world_right_wrist)),
    
    'weight_shift': (shoulder_mid_x - ankle_mid_x),
    
    'lateral_hip_position': left_mid_x,
    
    'spine_angle_1': calculate_angle(left_knee, left_hip, left_shoulder),
    'spine_angle_2': calculate_angle(right_knee, right_hip, right_shoulder),
    'spine_angle_3': left_shoulder[1] - left_hip[1]
    }
    
    return metrics


In [36]:
video = 0
cap = cv2.VideoCapture(video)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

counter = 0
stage = None
apex = 1000
prev_landmarks = None
prev_world_landmarks = None
top_rep_statistics = None
top_hip_position = 0
lateral_shift = 0
perSquatMetrics = {}
current_rep = {
    'max_depth': 0,
    'min_spine_angle': float('inf'),
    'max_lateral_shift': 0,
    'max_forward_shift': 0,
    'foot_distance': 0,
    'grip_width': 0,
    'knee_balance_bottom': None,
    'bottom_position_held': 0,  # frames spent at bottom position
    'knee_imbalance': 0
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Image because we want our image to be passed to MediaPipe in format to RGB (Default is of BGR)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection (Pose model)
        results = pose.process(image)
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            world_landmarks = results.pose_world_landmarks.landmark
            metrics = calculate_statistics(landmarks, world_landmarks)
            apex = min((world_landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y + world_landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y) / 2, apex) # Bottom of Squat
            
            # Display the metrics on the image
            print(metrics["knee_balance"])
            print(metrics["foot_distance"])
            cv2.putText(image, str(metrics["knee_balance"]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(metrics["knee_balance"][0] - metrics["knee_balance"][1]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, stage, (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 90), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Get Squat Metrics Per Frame and Per Rep
            if metrics["depth_left"] > 160 :
                if stage != "top rep":
                    if counter > 0:
                        perSquatMetrics[counter] = current_rep.copy()
                        current_rep = {
                            'max_depth': float('inf'),
                            'min_spine_angle': float('inf'),
                            'max_lateral_shift': 0,
                            'max_forward_shift': 0,
                            'foot_distance': 0,
                            'grip_width': 0,
                            'knee_balance_bottom': None,
                            'bottom_position_held': 0,  # frames spent at bottom position
                            'hips_below_knees': False,
                            'knee_imbalance': 0
                        }
                stage = "top rep"
                top_hip_position = metrics["lateral_hip_position"]
                lateral_shift = 0
            if metrics["depth_left"] < 160 and stage == 'top rep':
                stage = "bottom rep"
                counter += 1
            if stage == "bottom rep":
                current_rep['max_depth'] = min(current_rep['max_depth'], metrics["depth_left"])
                current_rep['min_spine_angle'] = min(current_rep['min_spine_angle'], metrics["spine_angle_3"])
                lateral_shift = (top_hip_position - metrics["lateral_hip_position"]) * 100
                current_rep['max_lateral_shift'] = lateral_shift if abs(lateral_shift) > abs(current_rep['max_lateral_shift']) else current_rep['max_lateral_shift']
                current_rep['max_forward_shift'] = metrics['weight_shift'] if abs(metrics['weight_shift']) > abs(current_rep['max_forward_shift']) else current_rep['max_forward_shift']
                current_rep['foot_distance'] = max(current_rep['foot_distance'], metrics["foot_distance"])
                current_rep['grip_width'] = max(current_rep['grip_width'], metrics["grip_width"])
                knee_imbalance = metrics["knee_balance"][0] - metrics["knee_balance"][1]
                current_rep['knee_imbalance'] = knee_imbalance if abs(knee_imbalance) > abs(current_rep['knee_imbalance']) else current_rep['knee_imbalance']

                if metrics["hips_below_knees"]:
                    current_rep['hips_below_knees'] = True
                    
                # Track knee balance at bottom position
                if metrics["depth_left"] < 120:  # Deep squat position
                    current_rep['bottom_position_held'] += 1

                    if current_rep['knee_balance_bottom'] is None:
                        current_rep['knee_balance_bottom'] = metrics["knee_balance"]
                
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        prev_landmarks = landmarks
        prev_world_landmarks = prev_world_landmarks

    cap.release()
    cv2.destroyAllWindows()
    
    

I0000 00:00:1740249628.479840 84893243 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740249628.573686 84938469 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740249628.586325 84938479 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[121.60261668374102, 142.29470039905672]
0.3077325815536637
[121.37107813430447, 133.21640577165104]
0.303681455917162
[122.32581277894118, 129.40510334432028]
0.305528436446967
[121.88113243734406, 127.29632854908203]
0.3051234861882855
[121.40587717016015, 126.26125663297212]
0.30500785443258666
[121.71998228090175, 125.84958993447495]
0.3047373102245504
[122.17881310741313, 125.4356837942735]
0.3031177142195115
[122.2454698497153, 124.67591872196267]
0.3020599012021167
[122.55985019537647, 123.48213955229623]
0.30241847646050896
[122.8802411959065, 123.48480148782161]
0.3020856258785486
[122.9281260894019, 123.67475856863658]
0.30123196340962477
[123.05612782967944, 123.51525996669328]
0.2997403896037207
[123.32227459773463, 121.92345604755403]
0.29736907838542287
[123.75402903501683, 120.71872551692145]
0.2954839469691979
[123.77467983842601, 120.12777228324049]
0.29380073173636806
[123.29073567825438, 119.93197515891201]
0.2927073763297757
[123.36047847105905, 119.48945429643162]


In [37]:
print("\nSquat Analysis:")
for rep_num, metrics in perSquatMetrics.items():
    print(f"\nRep {rep_num}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")


Squat Analysis:

Rep 1:
max_depth: 0
min_spine_angle: -0.9746242463588715
max_lateral_shift: -0.2142488956451416
max_forward_shift: -0.509849488735199
foot_distance: 0.3664311271471583
grip_width: 0.03831730499098335
knee_balance_bottom: None
bottom_position_held: 0
knee_imbalance: -7.49433539157782

Rep 2:
max_depth: 136.63313232982063
min_spine_angle: -0.9953292310237885
max_lateral_shift: 0.6157457828521729
max_forward_shift: -0.4718143716454506
foot_distance: 0.3835011337539179
grip_width: 0.037577003175207534
knee_balance_bottom: None
bottom_position_held: 0
hips_below_knees: False
knee_imbalance: 22.348891169139947

Rep 3:
max_depth: 128.19364355200534
min_spine_angle: -0.9606574475765228
max_lateral_shift: -1.7336726188659668
max_forward_shift: -0.6698592826724052
foot_distance: 0.47090339303681733
grip_width: 0.03558952568401609
knee_balance_bottom: None
bottom_position_held: 0
hips_below_knees: False
knee_imbalance: -36.89208184520905
